<a href="https://colab.research.google.com/github/YY2mk/KT_aivle/blob/master/DeepLearning/DX_%EB%94%A5%EB%9F%AC%EB%8B%9D%20%EC%8B%AC%ED%99%94_4%EC%9D%BC%EC%B0%A8%20%EC%8B%A4%EC%8A%B5%EC%9E%90%EB%A3%8C/05_%EC%96%B8%EC%96%B4%EB%AA%A8%EB%8D%B8_%EC%9D%B4%ED%95%B4_tokeninzing_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **언어 모델 이해**

# 1.환경준비

## (1) 라이브러리 설치

In [ ]:
# !pip install transformers
# colab에서는 이미 설치되어 있음

## (2) 라이브러리 Import

In [1]:
from transformers import pipeline

# 2.언어모델 사용 예제 코드 확인해보기

## (1) 전체 코드

In [6]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

#  Load Model and Tokenize
tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/kobart-news")
model = BartForConditionalGeneration.from_pretrained("ainize/kobart-news")

# Encode Input Text
input_text = '''
국내 전반적인 경기침체로 상가 건물주의 수익도 전국적인 감소세를 보이고 있는 것으로 나타났다.
수익형 부동산 연구개발기업 상가정보연구소는 한국감정원 통계를 분석한 결과 전국 중대형 상가
순영업소득(부동산에서 발생하는 임대수입, 기타수입에서 제반 경비를 공제한 순소득)이 1분기 ㎡당
3만4200원에서 3분기 2만5800원으로 감소했다고 17일 밝혔다.
수도권, 세종시, 지방광역시에서 순영업소득이 가장 많이 감소한 지역은 3분기 1만3100원을 기록한 울산으로, 1분기 1만9100원 대비 31.4% 감소했다.
이어 대구(-27.7%), 서울(-26.9%), 광주(-24.9%), 부산(-23.5%), 세종(-23.4%), 대전(-21%), 경기(-19.2%), 인천(-18.5%) 순으로 감소했다.
지방 도시의 경우도 비슷했다.
경남의 3분기 순영업소득은 1만2800원으로 1분기 1만7400원 대비 26.4% 감소했으며
제주(-25.1%), 경북(-24.1%), 충남(-20.9%), 강원(-20.9%), 전남(-20.1%), 전북(-17%), 충북(-15.3%) 등도 감소세를 보였다.
조현택 상가정보연구소 연구원은 "올해 내수 경기의 침체된 분위기가 유지되며
상가, 오피스 등을 비롯한 수익형 부동산 시장의 분위기도 경직된 모습을 보였고
오피스텔, 지식산업센터 등의 수익형 부동산 공급도 증가해 공실의 위험도 늘었다"며
"실제 올 3분기 전국 중대형 상가 공실률은 11.5%를 기록하며 1분기 11.3% 대비 0.2% 포인트 증가했다"고 말했다.
그는 "최근 소셜커머스(SNS를 통한 전자상거래), 음식 배달 중개 애플리케이션, 중고 물품 거래 애플리케이션 등의
사용 증가로 오프라인 매장에 영향을 미쳤다"며 "향후 지역, 콘텐츠에 따른 상권 양극화 현상은 심화될 것으로 보인다"고 덧붙였다.
'''

input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate Summary Text Ids
summary_text_ids = model.generate(
    input_ids=input_ids,
    bos_token_id=model.config.bos_token_id,
    eos_token_id=model.config.eos_token_id,
    length_penalty=2.0,
    max_length=142,
    min_length=56,
    num_beams=4,
)

# Decoding Text
print(tokenizer.decode(summary_text_ids[0], skip_special_tokens=True))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


국내 국내 전반적인 경기침체로 상가 건물주의 수익도 전국적인 감소세를 보이고 있는 것으로 나타났고 전국적인 감소세를 보이고 있는 것으로 나타났으며 국내 경기침체로 상가 건물주의 수익도 전국적인 감소세를 보이고 있는 것으로 나타났고 전국 중대형 상가 공실률은 1분기 11.3% 대비 0.2% 포인트 증가했으며 앞으로 지역, 콘텐츠에 따른 상권 양극화 현상은 심화될 것으로 보인다고 상가정보연구소 연구원은 말했다.


## (2) 코드를 나눠서 단계를 살펴봅시다.

* ① 모델, 토크나이저 다운로드

In [3]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

# Load Model and Tokenize
tokenizer = PreTrainedTokenizerFast.from_pretrained("ainize/kobart-news")
model = BartForConditionalGeneration.from_pretrained("ainize/kobart-news")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


* ② 입력 데이터 전처리(토크나이즈)

In [4]:
# 입력할 텍스트
input_text = '''
국내 전반적인 경기침체로 상가 건물주의 수익도 전국적인 감소세를 보이고 있는 것으로 나타났다.
수익형 부동산 연구개발기업 상가정보연구소는 한국감정원 통계를 분석한 결과 전국 중대형 상가
순영업소득(부동산에서 발생하는 임대수입, 기타수입에서 제반 경비를 공제한 순소득)이 1분기 ㎡당
3만4200원에서 3분기 2만5800원으로 감소했다고 17일 밝혔다.
'''

# 토크나이즈(encoding)
input_ids = tokenizer.encode(input_text, return_tensors="pt")

In [5]:
# 입력 토큰
input_ids

tensor([[    0,   230, 23436, 25623, 27091, 21067, 23895, 16442, 14453, 15342,
          9866, 14770, 14134, 15665, 14896, 16553, 14082, 14173, 15917, 18560,
         13679, 15821, 22164, 14451, 23895, 15107, 14867, 17047, 14188,  8995,
         22036, 14109, 15161, 22269, 14494, 14770, 22537, 13679, 23895,   230,
         11374, 18801, 16162,   239, 23625, 14030, 19668, 17249, 22540,   243,
         18453, 22540, 14030, 14060, 10773, 14051, 15541, 14061, 18010, 14408,
         16162, 15019, 20646,  1700, 17867,  9770,   230,   250, 10500,   251,
         18604, 15359, 20412, 18078,   252,   255, 14079, 15624, 15665, 14781,
         16826, 14278,     1]])

* ③ 예측(모델사용)


In [7]:
# 모델 사용
summary_text_ids = model.generate(
    input_ids=input_ids,                        # 입력으로 받는 데이터의 토큰
    bos_token_id=model.config.bos_token_id,     # Begin of String 문장이나 문단 등의 시작을 나타내는 특수한 토큰 ID
    eos_token_id=model.config.eos_token_id,     # End of String 문장이나 문단 등의 종료 나타내는 특수한 토큰 ID
    length_penalty=2.0,                         # 생성된 텍스트의 길이에 대한 패널티. 1< 더 긴 문장을 생성하려는 경향, 1 > 더 짧은 문장을 선호
    max_length=50,                              # 생성될 수 있는 최대 텍스트 길이(토큰 수)
    min_length=30,
    num_beams=4                                 # 빔 서치는 텍스트 생성 시 여러 가능성 있는 출력 중 최적의 결과를 찾기 위해 사용되는 기법. 이 클수록 더 많은 가능성을 탐색
)

In [8]:
# 요약된 문장 토큰
summary_text_ids

tensor([[    2,     0, 23436, 14454, 25623, 27091, 21067, 23895, 16442, 14453,
         15342,  9866, 14770, 14134, 15665, 14896, 16553, 14082, 14173, 19743,
         14297, 14454, 27091, 21067, 23895, 16442, 14453, 15342,  9866, 14770,
         14134, 15665, 14896, 16553, 14082, 14173, 19743,  9102, 14770, 12124,
         27091, 21067, 23895, 16442, 14453, 15342,  9866, 14770, 14134,     2]])

* ④ 결과 후처리


In [9]:
# 출력 토큰을 문장으로 변환(Decoding)
print(tokenizer.decode(summary_text_ids[0], skip_special_tokens=True))

국내 국내 전반적인 경기침체로 상가 건물주의 수익도 전국적인 감소세를 보이고 있는 것으로 나타났으며 국내 경기침체로 상가 건물주의 수익도 전국적인 감소세를 보이고 있는 것으로 나타났고 전국적 경기침체로 상가 건물주의 수익도 전국적인


# 3.토크나이즈

* 텍스트에서 토큰으로

### (1) 문자 토큰

* 텍스트 데이터셋

In [10]:
text_data = ["Tokenizers are one of the core components of the NLP.",
        "They serve one purpose: to translate text into data that can be processed by the model.",
        "Models can only process numbers, so tokenizers need to convert our text inputs to numerical data.",
        "In this section, we will explore exactly what happens in the tokenization pipeline.",
        "However, models can only process numbers, so we need to find a way to convert the raw text to numbers.",
        "That’s what the tokenizers do, and there are a lot of ways to go about this.",
        "The first type of tokenizer that comes to mind is word-based.",
        "It’s generally very easy to set up and use with only a few rules, and it often yields decent results.",
        "Character-based tokenizers split the text into characters, rather than words.",
        "The vocabulary is much smaller.",
        "There are much fewer out-of-vocabulary (unknown) tokens, since every word can be built from characters."]

* 어휘사전 만들기 : 문자별로 생성

In [11]:
# 문자별로 쪼개기
tokenized_text = []
for i in range(len(text_data)):
    tokenized_text = tokenized_text + list(text_data[i])

print(tokenized_text[:20])

# 각 문자에 인덱스(id) 부여하기
token2idx = {ch: idx for idx, ch in enumerate(sorted(set(tokenized_text)))}
print(token2idx)
print('vocab size : ',len(token2idx))

['T', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r', 's', ' ', 'a', 'r', 'e', ' ', 'o', 'n', 'e', ' ', 'o']
{' ': 0, '(': 1, ')': 2, ',': 3, '-': 4, '.': 5, ':': 6, 'C': 7, 'H': 8, 'I': 9, 'L': 10, 'M': 11, 'N': 12, 'P': 13, 'T': 14, 'a': 15, 'b': 16, 'c': 17, 'd': 18, 'e': 19, 'f': 20, 'g': 21, 'h': 22, 'i': 23, 'k': 24, 'l': 25, 'm': 26, 'n': 27, 'o': 28, 'p': 29, 'r': 30, 's': 31, 't': 32, 'u': 33, 'v': 34, 'w': 35, 'x': 36, 'y': 37, 'z': 38, '’': 39}
vocab size :  40


* 첫번째 문장을 id로 변환

In [12]:
input_text = text_data[0]
print('Input : ', input_text)
print(list(text_data[0]))

input_ids = [token2idx[token] for token in list(text_data[0])]
print(input_ids)

Input :  Tokenizers are one of the core components of the NLP.
['T', 'o', 'k', 'e', 'n', 'i', 'z', 'e', 'r', 's', ' ', 'a', 'r', 'e', ' ', 'o', 'n', 'e', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'c', 'o', 'r', 'e', ' ', 'c', 'o', 'm', 'p', 'o', 'n', 'e', 'n', 't', 's', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'N', 'L', 'P', '.']
[14, 28, 24, 19, 27, 23, 38, 19, 30, 31, 0, 15, 30, 19, 0, 28, 27, 19, 0, 28, 20, 0, 32, 22, 19, 0, 17, 28, 30, 19, 0, 17, 28, 26, 29, 28, 27, 19, 27, 32, 31, 0, 28, 20, 0, 32, 22, 19, 0, 12, 10, 13, 5]


### (2) 단어 토큰화

* 어휘사전 만들기 : 문자별로 생성

In [13]:
# 문자별로 쪼개기
tokenized_text = []
for i in range(len(text_data)):
    tokenized_text = tokenized_text + text_data[i].split()

print(tokenized_text[:2])

# 각 문자에 인덱스(id) 부여하기
token2idx = {ch: idx for idx, ch in enumerate(sorted(set(tokenized_text)))}
print(token2idx)
print('vocab size : ',len(token2idx))

['Tokenizers', 'are']
{'(unknown)': 0, 'Character-based': 1, 'However,': 2, 'In': 3, 'It’s': 4, 'Models': 5, 'NLP.': 6, 'That’s': 7, 'The': 8, 'There': 9, 'They': 10, 'Tokenizers': 11, 'a': 12, 'about': 13, 'and': 14, 'are': 15, 'be': 16, 'built': 17, 'by': 18, 'can': 19, 'characters,': 20, 'characters.': 21, 'comes': 22, 'components': 23, 'convert': 24, 'core': 25, 'data': 26, 'data.': 27, 'decent': 28, 'do,': 29, 'easy': 30, 'every': 31, 'exactly': 32, 'explore': 33, 'few': 34, 'fewer': 35, 'find': 36, 'first': 37, 'from': 38, 'generally': 39, 'go': 40, 'happens': 41, 'in': 42, 'inputs': 43, 'into': 44, 'is': 45, 'it': 46, 'lot': 47, 'mind': 48, 'model.': 49, 'models': 50, 'much': 51, 'need': 52, 'numbers,': 53, 'numbers.': 54, 'numerical': 55, 'of': 56, 'often': 57, 'one': 58, 'only': 59, 'our': 60, 'out-of-vocabulary': 61, 'pipeline.': 62, 'process': 63, 'processed': 64, 'purpose:': 65, 'rather': 66, 'raw': 67, 'results.': 68, 'rules,': 69, 'section,': 70, 'serve': 71, 'set': 72, '

* 첫번째 문장을 id로 변환

In [14]:
input_text = text_data[0]
print('Input : ', input_text)
print(text_data[i].split())

input_ids = [token2idx[token] for token in text_data[i].split()]
print(input_ids)

Input :  Tokenizers are one of the core components of the NLP.
['There', 'are', 'much', 'fewer', 'out-of-vocabulary', '(unknown)', 'tokens,', 'since', 'every', 'word', 'can', 'be', 'built', 'from', 'characters.']
[9, 15, 51, 35, 61, 0, 88, 73, 31, 101, 19, 16, 17, 38, 21]


### (3) 부분단어(Subwords) 토큰화

* 문자토큰과 단어토큰의 결합
* BERT 에서 사용되는 토크나이즈 WordPiece
* tokenization ==> token, #ization

In [15]:
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [16]:
text = text_data[0]
encoded_text = tokenizer(text)
print(text)
print(encoded_text)

Tokenizers are one of the core components of the NLP.
{'input_ids': [101, 19204, 17629, 2015, 2024, 2028, 1997, 1996, 4563, 6177, 1997, 1996, 17953, 2361, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


* input_ids : 각 토큰의 고유 정수값
* attention_mask :

In [17]:
# id를 다시 토큰으로 변
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

['[CLS]', 'token', '##izer', '##s', 'are', 'one', 'of', 'the', 'core', 'components', 'of', 'the', 'nl', '##p', '.', '[SEP]']


In [18]:
print(tokenizer.convert_tokens_to_string(tokens))

[CLS] tokenizers are one of the core components of the nlp. [SEP]


* 어휘 사전 크기

In [19]:
tokenizer.vocab_size

30522

* 모델의 최대 문맥 크기

In [20]:
tokenizer.model_max_length

512

* 모델 입력을 위한 필드 이름

In [21]:
tokenizer.model_input_names

['input_ids', 'attention_mask']

# 4.임베딩

* 자주 사용되는 Word2Vec 임베딩 벡터를 사용해 봅시다.

In [22]:
!pip install gensim
from gensim.models import Word2Vec

* 각 문장들을 토크나이즈 하기

In [23]:
token_sent = [tokenizer.convert_ids_to_tokens(tokenizer(text).input_ids) for text in text_data]
token_sent[0]

['[CLS]',
 'token',
 '##izer',
 '##s',
 'are',
 'one',
 'of',
 'the',
 'core',
 'components',
 'of',
 'the',
 'nl',
 '##p',
 '.',
 '[SEP]']

In [24]:
# Word2Vec 임베딩 생성
model = Word2Vec(sentences=token_sent, vector_size=30, window=5, min_count=1, workers=4)

# 토큰 'core'의 벡터 확인
token = 'components'
vector = model.wv[token]

print('index :', tokenizer([token]))
print('embedding vector : \n', vector.round(4))

index : {'input_ids': [[101, 6177, 102]], 'attention_mask': [[1, 1, 1]]}
embedding vector : 
 [ 0.0286 -0.0308  0.0243  0.0179  0.0044 -0.0173 -0.0139 -0.0112  0.0053
  0.0053  0.0247  0.0332  0.0296 -0.0134  0.0321 -0.0021  0.0163  0.0085
 -0.0021  0.0123 -0.0177 -0.0193 -0.0254  0.0065  0.0218  0.003   0.0043
  0.0106  0.0271 -0.0257]
